# Particle Swarm Optimization

The Particle swarm optimization(PSO) is an algorithm that was inspired in nature. The main idea is to mimic the behavior of the flock of birds or school of fish.

The particles in swarm doesn't'explore the fully search space, however, find an optimized solution. 

This method has random values ​​to perform the search, so it is a stochastic method

PSO Workflow:
1. Initialize particles
2. Evaluate fitness using the objective function
3. Update personal best position and score
4. Update global best position and score
5. (optional) Ends the search if it finds the best value that is already known and the position is unknown.
6. Update Velocity
7. Update Position
8. Repeat step 1 to 7 until the iterations finish



![Particle1](img/Particle_1.jpg)


## Move particle

To move particle, we will need to calculate the Inertia($\ I$), the Cognitive Component ($\ C$), and the Social Component($\ S$).

$\ I = w * V_{i}$

$\ C = c_{1} * rand[0, 1] * (X_{Best} - X_{Current})$

$\ S = c_{2} * rand[0, 1] * (X_{GlobalBest} - X_{Current})$

- $\ V_{i}$: velocity
- $\ w$: Inertia weight
- $\ c_{1}, c_{2}$: Constants
- $\ X_{Current}$: Current position
- $\ X_{Best}$: Particle best position
- $\ X_{GlobalBest}$: Global best position

The new velocity will be:

$\ V_{i} = I + C + S$

Then we can move the particle

$\ P_{new} = P_{current} + V $

We will try to optimize the following function:


$\ f(x, y) = x^2 + y^2 $

In [1]:
import numpy as np
from utils.plot_utils import plot_surface

In [2]:
bounds = [
    [-1, 1],
    [-1, 1],
]


def objective_function(vector: np.ndarray):
    return sum(vector ** 2)

In [3]:
plot_surface(objective_function, bounds)

# PSO
## Particles
We will need to initialize the particle position in a random position, and set best position with initial position.

The initial velocity vector will be initialized with zeros same length of the position.

```py
class Particle:

    position: NDArray[np.float64]
    score: float

    best_position: NDArray[np.float64]
    best_score: float

    velocity: NDArray[np.float64]

    def __init__(self, bound: Bound) -> None:

        self.position = np.array([np.random.uniform(b[0], b[1]) for b in bound])
        self.best_position = self.position
        self.velocity = np.zeros(len(bound))
        
        self.best_score = float('inf')
        self.score = float('inf')
```

In [4]:
from py_pso.particle import Particle

In [5]:
n_particles = 3

w=.5
c1 = .8
c2 = .9

In [6]:
particles = [Particle(bounds) for _ in range(n_particles)]
particles

[<Particle Current Position:                   (0.87, 0.98) | Best Position                   (0.87, 0.98) | Velocity:                   (0.00, 0.00) | Best Score:        inf>,
 <Particle Current Position:                  (0.51, -0.12) | Best Position                  (0.51, -0.12) | Velocity:                   (0.00, 0.00) | Best Score:        inf>,
 <Particle Current Position:                   (0.27, 0.81) | Best Position                   (0.27, 0.81) | Velocity:                   (0.00, 0.00) | Best Score:        inf>]

## Calculate the Scores

Using the current position, we will need to calculate the score for each particle. If the score is lower than the best score, we need to set this position as best position of the particle. We need to do the same thing for global best. 

In [7]:
global_best_score = float('inf')
global_best_position = None

In [8]:
for p in particles:
    score = objective_function(p.position)
    p.score = score

    if p.score < p.best_score:
        p.best_score = p.score
        p.best_position = p.position

    if p.score < global_best_score:
        global_best_score = p.score
        global_best_position = p.position

## Moving the particle

After calculating the particle and the best global position, we will need to calculate the velocity and move the particle using the equation we saw earlier. We will repeat the previews cell and the next cell until finish the iterations, or hit a stop condition when we can define.

In [9]:
for p in particles:
    inertia = (w * p.velocity)
    cognitive_component = c1 * np.random.rand() * (p.best_position - p.position)
    social_component = c2 * np.random.rand() * (global_best_position - p.position)

    p.velocity = inertia + cognitive_component + social_component
    p.move()

particles

[<Particle Current Position:                   (0.72, 0.52) | Best Position                   (0.87, 0.98) | Velocity:                 (-0.15, -0.46) | Best Score: 1.711496009925544>,
 <Particle Current Position:                  (0.51, -0.12) | Best Position                  (0.51, -0.12) | Velocity:                   (0.00, 0.00) | Best Score: 0.27619001164837065>,
 <Particle Current Position:                   (0.32, 0.63) | Best Position                   (0.27, 0.81) | Velocity:                  (0.05, -0.18) | Best Score: 0.7335156573893787>]

# More complex function

In [10]:
from py_pso.pso import PSO

from utils.functions import bird_function, bird_function_bounds

In [11]:
def objective_function(vector: np.ndarray):
    return bird_function(vector[0], vector[1])
    

bounds = bird_function_bounds

plot_surface(objective_function, bounds)

In [12]:

c1 = .8
c2 = .9
w = .5

n_iter = 1000
n_particles = 30

optimizer = PSO(objective_function=objective_function, n_particles=n_particles, bound=bounds, c1=c1, c2=c2, w=w)

best, score, hist = optimizer.optimize(iteration=n_iter, seed=1234, return_hist=True)

In [13]:
# The best score of bird function
bird_function_global_best = -106.7645367

print(best, score)
print(f'Error: {score - bird_function_global_best}')

[4.69365649 3.16032514] -106.78773368604186
Error: -0.023196986041867262


In [14]:


plot_surface(objective_function, bounds=bounds, particles=optimizer.particles)
